In [1]:
# 导入必要的库
import torch
import time
import matplotlib.pyplot as plt
from transformers import AutoTokenizer, AutoModelForCausalLM
import numpy as np
from typing import Optional, Tuple
import gc

# 设置随机种子确保结果可重现
torch.manual_seed(42)
np.random.seed(42)

print("库导入完成！")

c:\Users\k\.conda\envs\modelscope\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


库导入完成！


## 1. 加载本地Qwen2.5模型

我们将从您指定的本地路径加载Qwen2.5-0.5B-Instruct模型。

In [3]:
# 模型路径
model_path = r"C:\Users\k\Desktop\BaiduSyncdisk\baidu_sync_documents\hf_models\Qwen2.5-0.5B-Instruct"

# 加载tokenizer和模型
print("正在加载模型...")
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)

# 设置pad_token
if tokenizer.pad_token is None:
    print("未设置pad_token，使用eos_token作为pad_token")
    tokenizer.pad_token = tokenizer.eos_token

print(f"模型加载完成！")
print(f"模型参数量: {model.num_parameters():,}")
print(f"模型类型: {model.config.model_type}")
print(f"模型词汇表大小: {len(tokenizer)}")
print(f"模型设备: {next(model.parameters()).device}")

正在加载模型...
模型加载完成！
模型参数量: 494,032,768
模型类型: qwen2
模型词汇表大小: 151665
模型设备: cpu


In [10]:
# 测试无KV Cache的推理
test_prompt = "人工智能的未来发展趋势是"


In [ ]:
input_ids = tokenizer.encode(test_prompt, return_tensors="pt").to(model.device)
input_ids # 生成的输入ID

tensor([[104455,   9370, 100353, 108616,  20412]])

In [ ]:
with torch.no_grad():
    outputs = model(input_ids)
    # 打印下output的内容
    print(f"outputs类型: {type(outputs)}") # transformers.modeling_outputs.CausalLMOutputWithPast
    print(f"outputs内容: {outputs}") # 输出整个outputs对象的内容
    # 打印下outputs的其他属性，比如past_key_values,
    print(f"past_key_values类型: {type(outputs.past_key_values)}") # transformers.modeling_outputs.CausalLMOutputWithPast
    print(f"past_key_values内容: {outputs.past_key_values}") # 输出past_key_values的内容
    print("---" * 20)

outputs类型: <class 'transformers.modeling_outputs.CausalLMOutputWithPast'>
outputs内容: CausalLMOutputWithPast(loss=None, logits=tensor([[[ 5.2070,  9.7266,  3.8828,  ..., -2.3438, -2.3438, -2.3438],
         [ 5.8750, 13.0781,  5.7617,  ..., -2.7461, -2.7461, -2.7461],
         [10.0234,  9.0781,  4.7422,  ..., -4.0195, -4.0195, -4.0195],
         [ 9.7344,  8.6875,  6.6406,  ..., -3.9551, -3.9551, -3.9551],
         [ 5.9961, 11.2500,  7.0625,  ..., -4.1055, -4.1055, -4.1055]]],
       dtype=torch.float16), past_key_values=<transformers.cache_utils.DynamicCache object at 0x0000022A9639D360>, hidden_states=None, attentions=None)
past_key_values类型: <class 'transformers.cache_utils.DynamicCache'>
past_key_values内容: <transformers.cache_utils.DynamicCache object at 0x0000022A9639D360>
------------------------------------------------------------


In [ ]:
with torch.no_grad():
    outputs = model(input_ids)
    print(f"输出logits形状: {outputs.logits.shape}") # torch.Size([1, 5, 151936]),意思是批次大小为1，序列长度为5，词汇表大小为151936
    print(f"输出logits类型: {outputs.logits.dtype}") # torch.float16
    print(f"输出logits设备: {outputs.logits.device}") # cpu
    print(f"输出logits内容: {outputs.logits}") # 输出logits内容，这个内容指的是每个token的logits分数
    print(f"输出logits的第一个token的logits: {outputs.logits[0, 0, :10]}") # 输出第一个token的前10个logits分数, 这可以帮助我们理解模型对第一个token的预测
    print(f"输出logits的最后一个token的logits: {outputs.logits[0, -1, :10]}") # 输出最后一个token的前10个logits分数, 这可以帮助我们理解模型对最后一个token的预测
    # logits 是一个三维张量，形状为 (batch_size, sequence_length, vocab_size)，
    # 打印第一个维度
    print(f"输出logits的第一个维度: {outputs.logits.shape[0]}") # 输出logits的第一个维度，表示批次大小
    print(f"输出logits的第二个维度: {outputs.logits.shape[1]}") # 输出logits的第二个维度，表示序列长度
    print(f"输出logits的第三个维度: {outputs.logits.shape[2]}") # 输出logits的第三个维度，表示词汇表大小
    # token的id在矩阵的最后一个维度
    next_token_id = torch.argmax(outputs.logits[:, -1, :], dim=-1) # 获取下一个token的ID
    print("___" * 20)
    print(f"下一个token的ID: {next_token_id}") # 输出下一个token的ID，这个ID是模型预测的下一个token
   


输出logits形状: torch.Size([1, 5, 151936])
输出logits类型: torch.float16
输出logits设备: cpu
输出logits内容: tensor([[[ 5.2070,  9.7266,  3.8828,  ..., -2.3438, -2.3438, -2.3438],
         [ 5.8750, 13.0781,  5.7617,  ..., -2.7461, -2.7461, -2.7461],
         [10.0234,  9.0781,  4.7422,  ..., -4.0195, -4.0195, -4.0195],
         [ 9.7344,  8.6875,  6.6406,  ..., -3.9551, -3.9551, -3.9551],
         [ 5.9961, 11.2500,  7.0625,  ..., -4.1055, -4.1055, -4.1055]]],
       dtype=torch.float16)
输出logits的第一个token的logits: tensor([5.2070, 9.7266, 3.8828, 0.4221, 3.5195, 5.7695, 4.5703, 8.4062, 8.5625,
        6.0820], dtype=torch.float16)
输出logits的最后一个token的logits: tensor([ 5.9961, 11.2500,  7.0625,  6.9805,  3.8594,  5.1445,  8.9609, 14.9141,
         6.1797,  7.1914], dtype=torch.float16)
输出logits的第一个维度: 1
输出logits的第二个维度: 5
输出logits的第三个维度: 151936
输出logits的最后一个token的ID: 2130
____________________________________________________________
下一个token的ID: tensor([2130])


In [35]:
# 遍历第二个维度
with torch.no_grad():
    outputs = model(input_ids)
    print(f"输出logits形状: {outputs.logits.shape}")
    
    # 遍历序列中每个位置，获取预测的下一个token ID
    for i in range(outputs.logits.shape[1]):
        # 获取第i个位置对下一个token的预测
        predicted_token_id = outputs.logits[0, i, :].argmax()
        print(f"第{i}个位置预测的下一个token ID: {predicted_token_id}")
        
        # 如果想看对应的token文本
        predicted_token_text = tokenizer.decode([predicted_token_id])
        print(f"第{i}个位置预测的下一个token: '{predicted_token_text}'")

输出logits形状: torch.Size([1, 5, 151936])
第0个位置预测的下一个token ID: 9370
第0个位置预测的下一个token: '的'
第1个位置预测的下一个token ID: 2073
第1个位置预测的下一个token: '“'
第2个位置预测的下一个token ID: 102021
第2个位置预测的下一个token: '是什么'
第3个位置预测的下一个token ID: 102021
第3个位置预测的下一个token: '是什么'
第4个位置预测的下一个token ID: 2130
第4个位置预测的下一个token: '____'



## Transformer模型的预测机制

在Transformer的**因果语言建模**（Causal Language Modeling）中，模型确实会为序列中的**每个位置**都预测下一个token。

### 具体来说：

````python
# 遍历第二个维度
with torch.no_grad():
    outputs = model(input_ids)
    print(f"输出logits形状: {outputs.logits.shape}")
    
    # 遍历序列中每个位置，获取预测的下一个token ID
    for i in range(outputs.logits.shape[1]):
        # 获取第i个位置对下一个token的预测
        predicted_token_id = outputs.logits[0, i, :].argmax()
        print(f"第{i}个位置预测的下一个token ID: {predicted_token_id}")
        
        # 解释每个位置的含义
        if i == 0:
            print(f"  → 这是模型看到第1个token后，预测第2个token")
        elif i == outputs.logits.shape[1] - 1:
            print(f"  → 这是模型看到完整输入后，预测下一个新token (用于生成)")
        else:
            print(f"  → 这是模型看到前{i+1}个token后，预测第{i+2}个token")
        
        # 如果想看对应的token文本
        predicted_token_text = tokenizer.decode([predicted_token_id])
        print(f"第{i}个位置预测的下一个token: '{predicted_token_text}'")
        print("-" * 50)
````

### 为什么是这样设计的？

1. **训练时的需要**：
   - 在训练过程中，模型需要学习在看到部分序列时预测下一个token
   - 这样可以从一个序列中获得多个训练样本

2. **注意力掩码的作用**：
   - 第i个位置只能看到前i个token（包括自己）
   - 不能看到后面的token（因果性约束）

3. **并行训练**：
   - 可以同时计算所有位置的预测，提高训练效率

### 在例子中：

如果输入是"人工智能的未来发展趋势是"（5个token），那么：
- 位置0：看到"人工" → 预测下一个token
- 位置1：看到"人工智能" → 预测下一个token  
- 位置2：看到"人工智能的" → 预测下一个token
- 位置3：看到"人工智能的未来" → 预测下一个token
- 位置4：看到"人工智能的未来发展趋势是" → 预测下一个token（这个最重要！）

**在实际生成时，我们只关心最后一个位置的预测**，因为那是基于完整输入的预测结果。

这就是为什么在生成代码中我们用 `outputs.logits[:, -1, :]` 来获取最后一个位置的预测！